# Read/transform in Data

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os

path_to_project = os.path.abspath(os.path.join(os.getcwd(), '../'))    
sys.path.insert(1, os.path.join(path_to_project))

In [2]:
import itertools
import pandas as pd
from sklearn.preprocessing import StandardScaler
from src.data_dict import *
from src.utils import one_hot
from src.directory import data_dir, NHANES_preprocessed_filename
from src.estimators import aipw_estimator, unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner

In [3]:
# load df
NHANES_preprocessed_filepath = os.path.join(data_dir, NHANES_preprocessed_filename)
df = pd.read_csv(NHANES_preprocessed_filepath, index_col='SEQN')

In [4]:
# define relevant features
# features
z_col = light_col
t_col = sleep_deprivation_col
y_cols = [htn_col, diastolic_col, systolic_col]

# columns not to transform
all_cols = df.columns
untransformed_cols = [x for x in all_cols if x in [*y_cols, t_col, z_col]]

In [5]:
# prune df
df.dropna(how='any', inplace=True)
df = df.query(f'age >= {age_cutoff}')

## transform df
# apply scalers
scaler = StandardScaler()
numerical_transformation_cols = list(set(numerical_cols) - set(untransformed_cols))
df[numerical_transformation_cols] = scaler.fit_transform(df[numerical_transformation_cols])

# make z col binary (indicator of recommended max lux value during sleep)
light_cutoff = 1 # nightly minute-mean summed light exposure
df[z_col] = df[z_col].apply(lambda x: 1 if x <= light_cutoff else x)
df[z_col] = df[z_col].apply(lambda x: 0 if x > light_cutoff else x)

# one-hot encode multiclass categoricals
multiclass_cols = df[categorical_cols].columns[df[categorical_cols].nunique() > 2].tolist() 
categorical_transformation_cols = list(set(multiclass_cols) - set(untransformed_cols))
df = one_hot(df, categorical_transformation_cols)
df.columns = df.columns.str.replace('.0', '')

# get df as float
df = df.astype(float)

# except treatment
df[t_col] = df[t_col].astype(int)

# get covariates
x_cols = list(set(df.columns) - set([*y_cols, t_col, z_col]))

# update lists of variable type
all_cols = df.columns
categorical_cols = [x for x in df.columns if any([x.startswith(y) for y in categorical_cols])]

In [6]:
tau_pairs = [' vs '.join(x[::-1]) for x in itertools.combinations(df[t_col].unique().astype(str), 2)]

# Non-Deep Learning Estimators

In [7]:
estimators = [aipw_estimator, unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner]  
estimator_names = estimator_names = [x.__name__ for x in estimators]
index = pd.MultiIndex.from_product([estimator_names, tau_pairs, y_cols], names=['estimator', 'pair','outcome'])
tau_results = pd.DataFrame(index=index, columns=['tau', 'mse'])

In [8]:
for tau_estimator, outcome in itertools.product(estimators, y_cols):
    estimator_name = tau_estimator.__name__
    print(f'Estimating effect of {t_col} on {outcome} using {estimator_name}...')
    continuous_outcome = False if outcome == 'HTN' else True
    results = tau_estimator(df, 
                            treatment_var=t_col,
                            outcome_var=outcome, 
                            covariates=x_cols,
                            continuous_outcome=continuous_outcome)
    
    for pair in results.keys():
        tau = results[pair]['tau']
        tau_results.loc[(estimator_name, pair, outcome)] = tau

Estimating effect of sleep_deprivation on HTN using aipw_estimator...
	Outcome model score: 0.7864153906556733
Estimating effect of sleep_deprivation on DBP using aipw_estimator...
	Outcome model score: 0.28717906821838535
Estimating effect of sleep_deprivation on SBP using aipw_estimator...
	Outcome model score: 0.35531850697945544
Estimating effect of sleep_deprivation on HTN using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on DBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on SBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on HTN using ipw_estimator...
Estimating effect of sleep_deprivation on DBP using ipw_estimator...
Estimating effect of sleep_deprivation on SBP using ipw_estimator...
Estimating effect of sleep_deprivation on HTN using t_learner...
Estimating effect of sleep_deprivation on DBP using t_learner...
Estimating effect of sleep_deprivation on SBP using t_learner...
Estimating effect of sleep

In [9]:
tau_results

tau        mse
estimator               pair   outcome                      
aipw_estimator          1 vs 0 HTN      -0.032041  -0.032041
                               DBP      -0.815931  -0.815931
                               SBP      -0.996113  -0.996113
                        2 vs 0 HTN      -0.017358  -0.017358
                               DBP       -1.29625   -1.29625
                               SBP      -0.429396  -0.429396
                        2 vs 1 HTN       0.014683   0.014683
                               DBP      -0.480319  -0.480319
                               SBP       0.566717   0.566717
unadjusted_DM_estimator 1 vs 0 HTN      -0.079178  -0.079178
                               DBP       0.393923   0.393923
                               SBP      -2.830794  -2.830794
                        2 vs 0 HTN      -0.015747  -0.015747
                               DBP       1.239815   1.239815
                               SBP      -0.715358  -0.715358
                        2 vs 1 HTN       0.063431   0.063431
                               DBP       0.845891   0.845891
                               SBP       2.115436   2.115436
ipw_estimator           1 vs 0 HTN      -0.000416  -0.000416
                               DBP       3.223337   3.223337
                               SBP       5.460349   5.460349
                        2 vs 0 HTN       -0.14463   -0.14463
                               DBP     -15.169004 -15.169004
                               SBP     -26.883802 -26.883802
                        2 vs 1 HTN      -0.144213  -0.144213
                               DBP     -18.392341 -18.392341
                               SBP     -32.344151 -32.344151
t_learner               1 vs 0 HTN            0.0        0.0
                               DBP            0.0        0.0
                               SBP            0.0        0.0
                        2 vs 0 HTN            0.0        0.0
                               DBP            0.0        0.0
                               SBP            0.0        0.0
                        2 vs 1 HTN            0.0        0.0
                               DBP            0.0        0.0
                               SBP            0.0        0.0
s_learner               1 vs 0 HTN      -0.013812  -0.013812
                               DBP       -0.42152   -0.42152
                               SBP      -0.560957  -0.560957
                        2 vs 0 HTN       -0.01105   -0.01105
                               DBP       -0.81749   -0.81749
                               SBP      -0.327271  -0.327271
                        2 vs 1 HTN       0.002762   0.002762
                               DBP       -0.39597   -0.39597
                               SBP       0.233686   0.233686
x_learner               1 vs 0 HTN      -0.020919  -0.020919
                               DBP       -0.56257   -0.56257
                               SBP      -0.747388  -0.747388
                        2 vs 0 HTN      -0.002248  -0.002248
                               DBP      -0.465067  -0.465067
                               SBP       0.132967   0.132967
                        2 vs 1 HTN       0.018671   0.018671
                               DBP       0.097503   0.097503
                               SBP       0.880356   0.880356

# Deep learning estimators

In [10]:
import pandas as pd
from src.lightning_pipelines import DragonNet_training_pipeline

Seed set to 40


In [11]:
# set training variables
dataset_name = 'NHANES'
treatment_col = t_col
n_treatment_groups = df[treatment_col].nunique()
alpha = 1.0
beta = 1.0
input_features = x_cols
input_dim = len(input_features)
wandb_kwargs = dict(project_name='CPH_200B', 
                    wandb_entity='furtheradu', 
                    dir_path='notebooks/..',
                    offline=True)
max_epochs = 250

In [12]:
# set up results dataframe
target_regs = [True, False] # NOTE: target regularization is very sensitive to epsilon parameter -> leads to exploding loss
target_reg_str = {True:'_target_reg', False:''}
outcome_cols = y_cols
model_name = 'DragonNet'
learning_rates = {htn_col:1e-4, systolic_col:3e-4, diastolic_col:3e-4}
patience = 10

estimator_names = [f'{model_name}{target_reg_str[x]}' for x in target_regs]
index = pd.MultiIndex.from_product([estimator_names, tau_pairs, outcome_cols], names=['estimator', 'pair','outcome'])
dragon_tau_results = pd.DataFrame(index=index, columns=['tau', 'mse']).sort_index()

In [13]:
for target_reg, outcome_col in itertools.product(target_regs, outcome_cols):
    print(f'Training model {model_name} on {dataset_name} with target_reg == {target_reg} to predict CATE of {treatment_col} on {outcome_col}')    
    pipeline_out = DragonNet_training_pipeline(
        raw_data=df,
        model_name=model_name,
        treatment_col=treatment_col,
        outcome_col=outcome_col,
        input_features=input_features,
        dataset_name=dataset_name,
        alpha=alpha,
        beta=beta,
        target_reg=target_reg,
        wandb_kwargs=wandb_kwargs,
        n_treatment_groups=n_treatment_groups,
        max_epochs=max_epochs,
        learning_rate=learning_rates[outcome_col],
        patience=patience
    )

    trainer, model, datamodule = [pipeline_out[x] for x in ['trainer', 'model', 'datamodule']]
    
    print("Testing model")
    test_pred = trainer.test(model, datamodule)

    # log results to df
    estimator_name = f'{model_name}_{target_reg_str[target_reg]}'
    for pair in tau_pairs:
        dragon_tau_results.loc[(estimator_name, pair, outcome_col), 'tau'] = test_pred[0][f'test_tau_{pair}']
        dragon_tau_results.loc[(estimator_name, pair, outcome_col), 'mse'] = test_pred[0][f'test_outcome_model_mse']
        
    print(f'Finished training.\n\n')

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training model DragonNet on NHANES with target_reg == True to predict CATE of sleep_deprivation on HTN
Training model


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id 03ipfupp.


Processing NHANES Dataset: 2547it [00:00, 9840.70it/s] 
/home/hentipapai/miniconda3/envs/cph200b_project3/lib/python3.11/site-packages/pytorch_lightning/core/optimizer.py:378: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name  | Type           | Params | Mode 
-------------------------------------------------
0 | model | DragonNetTorch | 118 K  | train
-------------------------------------------------
118 K     Trainable params
0         Non-trainable params
118 K     Total params
0.474     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Testing model


Processing NHANES Dataset: 2547it [00:00, 12372.58it/s]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
           Test metric                     DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_dragon_net_loss              343.13868606988746
            test_loss                      343.138671875
     test_outcome_model_mse             342.12632066705334
test_targeted_regularization_loss               0.0
         test_tau_1 vs 0                -1.599974513053894
         test_tau_2 vs 0                -10.972472190856934
         test_tau_2 vs 1                -9.372498512268066
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Finished training.




In [14]:
dragon_tau_results.dropna(how='all')

tau         mse
estimator             pair   outcome                       
DragonNet__target_reg 1 vs 0 HTN       0.317338    0.397999
                      2 vs 0 HTN      -0.913264    0.397999
                      2 vs 1 HTN      -1.230602    0.397999
                      1 vs 0 DBP       0.280373  196.868244
                      2 vs 0 DBP      -2.122968  196.868244
                      2 vs 1 DBP      -2.403341  196.868244
                      1 vs 0 SBP       1.301621  360.762624
                      2 vs 0 SBP     -15.000551  360.762624
                      2 vs 1 SBP     -16.302174  360.762624
DragonNet_            1 vs 0 HTN       0.425045    0.263338
                      2 vs 0 HTN      -0.134726    0.263338
                      2 vs 1 HTN      -0.559771    0.263338
                      1 vs 0 DBP       8.361858  202.180345
                      2 vs 0 DBP      -2.898064  202.180345
                      2 vs 1 DBP     -11.259922  202.180345
                      1 vs 0 SBP      -1.599975  342.126321
                      2 vs 0 SBP     -10.972472  342.126321
                      2 vs 1 SBP      -9.372499  342.126321